<a href="https://colab.research.google.com/github/jaimiejl/neuroimaging_transcriptomics/blob/main/MWI_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install abagen

import abagen
from abagen import reporting
from statistics import mean, stdev
import pandas as pd
from scipy.stats.stats import pearsonr
from scipy import stats
from math import sqrt
import random
from statsmodels.stats.multitest import fdrcorrection
from functools import reduce

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 19.7 MB/s eta 0:00:00


<ipython-input-1-881e171ca1ba>:7: DeprecationWarning: Please use `pearsonr` from the `scipy.stats` namespace, the `scipy.stats.stats` namespace is deprecated.
  from scipy.stats.stats import pearsonr


In [ ]:
from google.colab import files # Load mwf masks & mwf data
files.upload()

Saving mwf_data.xlsx to mwf_data.xlsx


{'mwf_data.xlsx': b'PK\x03\x04\x14\x00\x06\x00\x08\x00\x00\x00!\x00b\xee\x9dh^\x01\x00\x00\x90\x04\x00\x00\x13\x00\x08\x02[Content_Types].xml \xa2\x04\x02(\xa0\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

In [ ]:
# Function to get the mean sample for a given mask, region, and subject
def get_mean_sample(mask_path, region_name, donors):
    # Call abagen function to get samples in the mask for the specified donor
    df, coords = abagen.get_samples_in_mask(mask=mask_path, donors=donors, probe_selection='corr_intensity')
    # Calculate the mean of the samples and append it as a new row
    df.loc[region_name, :] = df.mean()
    # Return the last row which contains the mean sample
    return df.iloc[-1]

# Dictionary mapping subject IDs to their respective names
subjects = {
    '9861': 'donor9861',
    '10021': 'donor10021',
    '12876': 'donor12876',
    '14380': 'donor14380',
    '15496': 'donor15496',
    '15697': 'donor15697'
}

# List of masks and their corresponding region names
masks_info = [
    ('/content/Body_of_corpus_callosum.nii.gz', 'cc_body'), # Enter path to respective mask
    ('/content/Cerebral_White_Matter_bin_95.nii.gz', 'wm'), # Enter path to respective mask
    ('/content/Hippocampus_bin_80.nii.gz', 'hippo'), # Enter path to respective mask
    ('/content/internal_capsule_posterior_limb_manual.nii.gz', 'ic_post'), # Enter path to respective mask
    ('/content/WM_Corticospinal_tract.nii.gz', 'corticospinal') # Enter path to respective mask
]

# Dictionary to store donor-specific DataFrames
donor_dataframes = {}

# Loop through each subject and process their data
for subject, subject_name in subjects.items():
    # Create an empty DataFrame to store subject-specific data
    subject_data = pd.DataFrame()
    # Loop through each mask and region to get mean samples
    for mask_path, region_name in masks_info:
        # Append mean sample data for each region to the subject-specific DataFrame
        subject_data[region_name] = get_mean_sample(mask_path, region_name, subject)
    # Store the subject-specific DataFrame in the donor-specific DataFrame dictionary
    donor_dataframes[subject_name] = subject_data

# Print donor_dataframes to verify the structure
print(donor_dataframes)

# Access individual donor DataFrames like donor_dataframes['donor9861'], etc.



Dataset created in /root/abagen-data/microarray



Downloaded 425988059 of 425988059 bytes (100.0%,    0.0s remaining) ...done. (60 seconds, 1 min)
Extracting data from /root/abagen-data/microarray/115d7432fc9ff261ad0e4d28eaad187e/normalized_microarray_donor9861/donor9861.zip..... done.
/usr/local/lib/python3.10/dist-packages/abagen/probes_.py:757: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  micro = micro.set_axis(symbols, axis=1, inplace=False)
/usr/local/lib/python3.10/dist-packages/abagen/probes_.py:757: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  micro = micro.set_axis(symbols, axis=1, inplace=False)
/usr/local/lib/python3.10/dist-packages/abagen/probes_.py:757: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False

Downloaded 400957002 of 400957002 bytes (100.0%,    0.0s remaining) ...done. (60 seconds, 0 min)
Extracting data from /root/abagen-data/microarray/f2c88d92c7acbba2d24a13349cab71ed/normalized_microarray_donor10021/donor10021.zip..... done.
/usr/local/lib/python3.10/dist-packages/abagen/probes_.py:757: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  micro = micro.set_axis(symbols, axis=1, inplace=False)
/usr/local/lib/python3.10/dist-packages/abagen/probes_.py:757: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  micro = micro.set_axis(symbols, axis=1, inplace=False)
/usr/local/lib/python3.10/dist-packages/abagen/probes_.py:757: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=Fal

Downloaded 166233851 of 166233851 bytes (100.0%,    0.0s remaining) ...done. (26 seconds, 0 min)
Extracting data from /root/abagen-data/microarray/e6f4cbce4d5f941b7b805be2308e2067/normalized_microarray_donor12876/donor12876.zip..... done.
/usr/local/lib/python3.10/dist-packages/abagen/probes_.py:757: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  micro = micro.set_axis(symbols, axis=1, inplace=False)
/usr/local/lib/python3.10/dist-packages/abagen/probes_.py:757: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  micro = micro.set_axis(symbols, axis=1, inplace=False)
/usr/local/lib/python3.10/dist-packages/abagen/probes_.py:757: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=Fal

Downloaded 241359585 of 241359585 bytes (100.0%,    0.0s remaining) ...done. (34 seconds, 0 min)
Extracting data from /root/abagen-data/microarray/d174218c74ec4d433c0bbac4fee02516/normalized_microarray_donor14380/donor14380.zip..... done.
/usr/local/lib/python3.10/dist-packages/abagen/probes_.py:757: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  micro = micro.set_axis(symbols, axis=1, inplace=False)
/usr/local/lib/python3.10/dist-packages/abagen/probes_.py:757: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  micro = micro.set_axis(symbols, axis=1, inplace=False)
/usr/local/lib/python3.10/dist-packages/abagen/probes_.py:757: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=Fal

Downloaded 216077630 of 216077630 bytes (100.0%,    0.0s remaining) ...done. (31 seconds, 0 min)
Extracting data from /root/abagen-data/microarray/2561858739a2d8c9a0d52bc38660d0a4/normalized_microarray_donor15496/donor15496.zip..... done.
/usr/local/lib/python3.10/dist-packages/abagen/probes_.py:757: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  micro = micro.set_axis(symbols, axis=1, inplace=False)
/usr/local/lib/python3.10/dist-packages/abagen/probes_.py:757: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  micro = micro.set_axis(symbols, axis=1, inplace=False)
/usr/local/lib/python3.10/dist-packages/abagen/probes_.py:757: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=Fal

Downloaded 230640408 of 230640408 bytes (100.0%,    0.0s remaining) ...done. (32 seconds, 0 min)
Extracting data from /root/abagen-data/microarray/dec750c1301d020337ca59ae44bb5ca1/normalized_microarray_donor15697/donor15697.zip..... done.
/usr/local/lib/python3.10/dist-packages/abagen/probes_.py:757: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  micro = micro.set_axis(symbols, axis=1, inplace=False)
/usr/local/lib/python3.10/dist-packages/abagen/probes_.py:757: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  micro = micro.set_axis(symbols, axis=1, inplace=False)
/usr/local/lib/python3.10/dist-packages/abagen/probes_.py:757: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=Fal

{'donor9861':               cc_body        wm     hippo   ic_post  corticospinal
gene_symbol                                                       
A1BG         0.607650  0.510020  0.431651  0.645076       0.529291
A1BG-AS1     0.566385  0.394555  0.169057  0.181718       0.288960
A2M          0.317239  0.517768  0.519464  0.670116       0.606003
A2ML1        0.619780  0.586028  0.587463  0.648498       0.539466
A3GALT2      0.458933  0.429408  0.632467  0.484405       0.519505
...               ...       ...       ...       ...            ...
ZXDC         0.572594  0.629230  0.644799  0.803209       0.589207
ZYG11B       0.553848  0.406087  0.377734  0.275724       0.355674
ZYX          0.465877  0.413901  0.572482  0.423160       0.527742
ZZEF1        0.424581  0.507463  0.655768  0.522850       0.568210
ZZZ3         0.643379  0.530199  0.533028  0.577345       0.579747

[14955 rows x 5 columns], 'donor10021':               cc_body        wm     hippo   ic_post  corticospinal
gene_sy

In [ ]:
# Assuming donor_dataframes is a dictionary with donor dataframes
donor_ids = ['donor9861', 'donor10021', 'donor12876', 'donor14380', 'donor15496', 'donor15697']

# Using the index of each dataframe to find the common genes
common_genes = reduce(lambda x, y: x.intersection(donor_dataframes[y].index), donor_ids, donor_dataframes[donor_ids[0]].index)

final_genes_df = pd.DataFrame({'gene': list(common_genes)})

final_gene_list = list(common_genes)


In [ ]:
cleaned_donors = {}
for donor in ['donor9861', 'donor10021', 'donor12876', 'donor14380', 'donor15496', 'donor15697']:
    cleaned_donors[donor] = donor_dataframes[donor].transpose().loc[:, final_gene_list]


In [ ]:
# Assuming 'final_gene_list' and 'cleaned_donors' are already defined
donor_pairs = [
    ('donor9861', 'donor10021', 'onetwo'),
    ('donor9861', 'donor12876', 'onethree'),
    ('donor9861', 'donor14380', 'onefour'),
    ('donor9861', 'donor15496', 'onefive'),
    ('donor9861', 'donor15697', 'onesix'),
    ('donor10021', 'donor12876', 'twothree'),
    ('donor10021', 'donor14380', 'twofour'),
    ('donor10021', 'donor15496', 'twofive'),
    ('donor10021', 'donor15697', 'twosix'),
    ('donor12876', 'donor14380', 'threefour'),
    ('donor12876', 'donor15496', 'threefive'),
    ('donor12876', 'donor15697', 'threesix'),
    ('donor14380', 'donor15496', 'fourfive'),
    ('donor14380', 'donor15697', 'foursix'),
    ('donor15496', 'donor15697', 'fivesix'),
]

# Initialize a dictionary with the correlation lists
correlations = {name: [] for _, _, name in donor_pairs}

# Compute correlations for each gene and append to the respective lists
for gene in final_gene_list:
    for donor1, donor2, name in donor_pairs:
        correlations[name].append(pearsonr(cleaned_donors[donor1][gene], cleaned_donors[donor2][gene])[0])

# Create the DataFrame
autocorrect_df = pd.DataFrame({'gene': final_gene_list, **correlations})

autocorrect_df


,gene,onetwo,onethree,onefour,onefive,onesix,twothree,twofour,twofive,twosix,threefour,threefive,threesix,fourfive,foursix,fivesix
0,A1BG,0.649006,-0.664251,0.501973,0.434980,0.110941,-0.406897,0.835611,0.943951,0.759039,0.083864,-0.092338,0.213366,0.928717,0.813672,0.898787
1,A1BG-AS1,0.836706,0.516457,0.652266,0.729326,0.761609,0.892248,0.946972,0.872152,0.928542,0.932065,0.701997,0.822280,0.907021,0.953147,0.916652
2,A2M,0.522256,0.410037,0.313913,0.706367,0.268374,0.989160,0.815956,0.940745,0.960176,0.820700,0.884323,0.988227,0.810330,0.826055,0.825600
3,A2ML1,0.198435,-0.040338,0.398009,-0.297755,0.674728,0.647696,0.618789,0.822337,0.005290,0.636861,0.385697,0.294338,0.202250,0.703308,-0.531231
4,A4GALT,0.198570,0.858923,0.506793,0.711234,0.601355,0.537408,0.890849,0.654841,-0.454081,0.766212,0.852541,0.361546,0.925921,-0.302094,-0.085636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14612,ZXDC,0.633273,0.155316,-0.311971,-0.584626,0.168652,-0.332344,0.328599,-0.460534,0.044934,-0.045820,-0.205966,-0.679743,-0.364139,-0.699448,0.380831
14613,ZYG11B,0.787311,-0.622680,0.755241,0.887837,0.770587,-0.141523,0.986733,0.909834,0.956431,-0.025019,-0.261579,-0.060124,0.919135,0.973935,0.967473
14614,ZYX,-0.437696,-0.202537,-0.293878,-0.231232,0.096403,0.852668,0.983566,0.941360,0.802620,0.846705,0.972161,0.919422,0.947095,0.866479,0.937463
14615,ZZEF1,-0.582888,0.699569,0.414053,0.600722,0.679689,-0.046621,0.434014,-0.148491,0.194206,0.710171,0.320769,0.854609,0.658554,0.895867,0.553488


In [ ]:
# Calculate the median of each row excluding the first column
row_medians = autocorrect_df.iloc[:, 1:].median(axis=1)

# Add median column to the DataFrame
autocorrect_df_with_median = pd.concat([autocorrect_df, row_medians.rename('median')], axis=1)


In [ ]:
final_median_pearsonr = autocorrect_df_with_median[autocorrect_df_with_median['median'] > 0.4][['gene', 'median']]
final_median_pearsonr.columns = ['gene', 'median_pearsonr']

donors = ['donor9861', 'donor10021', 'donor12876', 'donor14380', 'donor15496', 'donor15697']
donors_cleaned_postmedian = {donor: cleaned_donors[donor][final_median_pearsonr['gene']] for donor in donors}

In [ ]:
mwf = pd.read_excel('/content/mwf_data.xlsx') # mwf data

In [ ]:
# Function to compute values
def compute_values(region1, region2, mwf):
    mean1, mean2 = mean(mwf[region1]), mean(mwf[region2])
    stdev1, stdev2 = stdev(mwf[region1]), stdev(mwf[region2])
    return (mean2 - mean1) / sqrt((stdev1 ** 2 + stdev2 ** 2) / 2)

# Define the regions of interest and their corresponding names in 'mwf'
roi = ['ic_post', 'corticospinal', 'wm', 'cc_body', 'hippo']
mwf_columns = {
    'ic_post': 'Internal_capsule_posterior',
    'corticospinal': 'Corticospinal',
    'wm': 'White_matter',
    'cc_body': 'Corpus_callosum_body',
    'hippo': 'Hippocampus'
}

# Initialize DataFrame
mwf_df = pd.DataFrame(columns=['roi'] + list(roi))

# Populate DataFrame
for region in roi:
    mwf_df[region] = [compute_values(mwf_columns[region], mwf_columns[other_region], mwf) for other_region in roi]

mwf_df['roi'] = roi
mwf_df


,roi,ic_post,corticospinal,wm,cc_body,hippo
0,ic_post,0.000000,1.746630,4.490684,5.253575,6.083270
1,corticospinal,-1.746630,0.000000,3.115473,4.031770,5.064936
2,wm,-4.490684,-3.115473,0.000000,1.124793,2.762007
3,cc_body,-5.253575,-4.031770,-1.124793,0.000000,1.817414
4,hippo,-6.083270,-5.064936,-2.762007,-1.817414,0.000000


In [ ]:
def compute_values(region1, region2):
    mean1, mean2 = mean(region1), mean(region2)
    stdev1, stdev2 = stdev(region1), stdev(region2)
    return (mean2 - mean1) / sqrt((stdev1 ** 2 + stdev2 ** 2) / 2)

def make_matrix(ic_post, corticospinal, wm, cc_body, hippo):
    rois = [ic_post, corticospinal, wm, cc_body, hippo]
    roi_names = ['ic_post', 'corticospinal', 'wm', 'cc_body', 'hippo']

    data = {roi_name: [compute_values(roi, other_roi) for other_roi in rois] for roi, roi_name in zip(rois, roi_names)}

    return pd.DataFrame(data, index=roi_names)

# Example usage
# Assuming you have the data for ic_post, corticospinal, wm, cc_body, hippo
# matrix_df = make_matrix(ic_post, corticospinal, wm, cc_body, hippo)

def collect_expressions(gene, roi): #'donor9861', 'donor10021', 'donor12876', 'donor14380', 'donor15496', 'donor15697'
    # Assuming donor1, donor2, ... are defined somewhere accessible
    donors = [donors_cleaned_postmedian['donor9861'], donors_cleaned_postmedian['donor10021'], donors_cleaned_postmedian['donor12876'], donors_cleaned_postmedian['donor14380'], donors_cleaned_postmedian['donor15496'], donors_cleaned_postmedian['donor15697']]
    return [donor[gene][roi] for donor in donors]

# Example usage
# expressions = collect_expressions(some_gene, some_roi)


In [ ]:
gene_list = final_median_pearsonr['gene'].tolist()

Dictionary = {gene: make_matrix(collect_expressions(gene, 3),
                                collect_expressions(gene, 4),
                                collect_expressions(gene, 1),
                                collect_expressions(gene, 0),
                                collect_expressions(gene, 2))
              for gene in gene_list}


In [ ]:
# Extracting upper right-hand values from mwf_df
mwf_list = mwf_df.iloc[0].values.tolist()[2:] + \
           mwf_df.iloc[1].values.tolist()[3:] + \
           mwf_df.iloc[2].values.tolist()[4:] + \
           mwf_df.iloc[3].values.tolist()[5:]

# Initialize dictionaries for Pearson correlation coefficients and p-values
Dictionary_pearson = {}
Dictionary_pearson_pval = {}

# Iterate over each gene in the Dictionary
for key in Dictionary:
    gene_matrix = Dictionary[key]
    gene_exp_list = gene_matrix.iloc[0].values.tolist()[1:] + \
                    gene_matrix.iloc[1].values.tolist()[2:] + \
                    gene_matrix.iloc[2].values.tolist()[3:] + \
                    gene_matrix.iloc[3].values.tolist()[4:]
    r_value, p_value = pearsonr(mwf_list, gene_exp_list)
    Dictionary_pearson[key] = r_value
    Dictionary_pearson_pval[key] = p_value

# Create DataFrame from the results
df_pearson = pd.DataFrame({
    'gene': list(Dictionary_pearson.keys()),
    'pearson_r': list(Dictionary_pearson.values()),
    'p_val': list(Dictionary_pearson_pval.values())
})

# Adjust p-values and sort the DataFrame
df_pearson['adj_p_val'] = fdrcorrection(df_pearson['p_val'])[1]
df_pearson_sorted = df_pearson.sort_values('pearson_r', ascending=False)

df_pearson_sorted


,gene,pearson_r,p_val,adj_p_val
3013,FAM173B,0.990825,3.066012e-08,0.000340
1469,CD300C,0.985040,2.152416e-07,0.001192
8107,RRAGC,0.982775,3.772606e-07,0.001347
6993,PLEKHG1,0.981394,5.127198e-07,0.001347
2900,ETFDH,0.977560,1.079668e-06,0.001447
...,...,...,...,...
5397,MARK2,-0.976159,1.373452e-06,0.001447
4003,HIST1H4K,-0.976631,1.268621e-06,0.001447
5380,MAPKBP1,-0.977475,1.096013e-06,0.001447
2344,DGAT1,-0.978388,9.299651e-07,0.001447
